In [1]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import fuzzymatcher
pd.options.display.max_rows = 250

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


# Script Testing
## Vehicles - Help
* Can't get grouping to work as an inner function

In [2]:
df = data_prep.load_grantprojects()
vehicles = data_prep.load_vehiclesdata()
organizations= data_prep.load_organizations_data()

/home/jovyan/data-analyses/5311/data_prep.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
# Grouping vehicle types together: not working in function
Automobiles = ['Automobile','Automobiles (Service)','Sports Utility Vehicle']
Bus = ['Bus','Over-the-road Bus']
Vans = ['Van','Trucks and other Rubber Tire Vehicles (Service)','Minivan','Cutaway',]

In [4]:
def replace_modes(row):
        if row.vehicle_type in Automobiles:
            return "Rail"
        elif row.vehicle_type in Bus:
            return "Bus"
        else:
            return "Vans"
vehicles["vehicle_groups"] = vehicles.apply(lambda x: replace_modes(x), axis=1)

# Basic Facts about our data

### Vehicles 
* Only rural reporters included, 52 of them

In [5]:
f'{len(vehicles)} rows in NTD vehicles'

'140 rows in NTD vehicles'

In [6]:
f'{vehicles.agency.nunique()} total unique agencies.'

'52 total unique agencies.'

### Black Cat data for only 5311 programs
* Most funding program is just section 5311
* What differentiates them?
* In my 5311 First look notebook, only 23 unique agencies left after filtering for only bus related projects

In [7]:
df.funding_program.value_counts()

Section 5311       590
Section 5311(f)    109
5311(f) Cont        41
CMAQ (FTA 5311)     31
5311(f) Round 2     27
Name: funding_program, dtype: int64

In [8]:
f'a total of {df.organization_name.nunique()} unique agencies after filtering for 5311'

'a total of 88 unique agencies after filtering for 5311'

In [9]:
df.head(2)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status
0,2011,Section 5311,CA-18-X047 | 0012000083,2016,City of Chowchilla,BCG0000228,Operating Assistance,300902,64BO17-00368,53221.0,114511.0,53221.00,0.00,0,Open
1,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000283,Buy <30-Ft Bus For Expansion,111304,64BC17-00408,110663.0,110663.0,101352.02,9310.98,0,Open


In [10]:
#subset for crosswalk
crosswalk_5311 = df[['organization_name']]

In [11]:
crosswalk_5311 = crosswalk_5311.drop_duplicates()

In [12]:
crosswalk_5311.sort_values('organization_name')

,organization_name
114,Alpine County Community Development
115,Amador Transit
25,Butte County Association of Governments/ Butte...
26,Calaveras County Public Works
1029,Calaveras Transit Agency
11,City of Arcata
29,City of Arvin
30,City of Auburn
31,City of California City
0,City of Chowchilla


### Organization Data
* Has NTD ID, ITP ID, and GTFS information
* After deleting all rows that do not have NTP IDs, rows went from 781 rows to 199
* So only 199 agencies have NTD IDS in the organizations data set?

In [13]:
organizations2 = organizations.dropna(subset=['ntd_id'])

In [14]:
f'a total of {organizations.agency.nunique()} unique agencies after filtering for 5311, keeping every single agency'

'a total of 781 unique agencies after filtering for 5311, keeping every single agency'

In [15]:
f'a total of {organizations2.agency.nunique()} unique agencies after filtering for 5311 & only keeping agencies with NTD IDS'

'a total of 199 unique agencies after filtering for 5311 & only keeping agencies with NTD IDS'

In [16]:
organizations_crosswalk = organizations2[['agency','ntd_id', 'itp_id']]

# Crosswalk 
### Look at unique rural agencies in vehicles

In [17]:
#subset for crosswalk
vehicles_crosswalk = vehicles[['agency','ntd_id']]

In [18]:
#drop duplicates for crosswalk
vehicles_crosswalk = vehicles_crosswalk.drop_duplicates()

In [19]:
vehicles_crosswalk.sort_values('agency')

,agency,ntd_id
6668,Alpine County Local Transportation Commission,9R02-91116
4637,Amador Regional Transit System,9R02-91000
5797,Calaveras Transit Agency,9R02-99442
6451,City of Arcata,9R02-91018
5788,City of Arvin,9R02-91027
6001,City of Auburn,9R02-91032
6460,City of California City,9R02-91111
6665,"City of Chowchilla, dba: Chowchilla Area Transit",9R02-91071
5786,"City of Corcoran, dba: Corcoran Area Transit",9R02-91002
6004,City of Dinuba,9R02-91040


##  Left join on "organizations" with GTFS with vehicles
* Using left join, organization crosswalk on the L. Vehicles on the R. 
* Only 1/2 (26) of agencies in vehicles (total 52) appears in Cal ITP
* 173 of the records in organizations have Cal ITP ids but aren't in the vehicles dataset, which makes sense since these are tons of non-rural reporters in the organizations data set

In [20]:
df_joined1 = organizations_crosswalk.merge(vehicles_crosswalk.drop_duplicates(), on=['agency', 'ntd_id'],  how='left', validate = "1:1", indicator=True)

In [21]:
df_joined1._merge.value_counts()

left_only     173
both           26
right_only      0
Name: _merge, dtype: int64

### Filtering out for only agencies with both...

In [22]:
df_joined_both = df_joined1.loc[df_joined1['_merge'] == 'both']

In [23]:
df_joined_both=df_joined_both.drop_duplicates()

In [24]:
df_joined_both

,agency,ntd_id,itp_id,_merge
3,Amador Regional Transit System,9R02-91000,11.0,both
17,City of Arcata,9R02-91018,18.0,both
19,City of Arvin,9R02-91027,21.0,both
21,City of Auburn,9R02-91032,23.0,both
31,City of California City,9R02-91111,51.0,both
45,City of Dinuba,9R02-91040,93.0,both
57,City of Guadalupe,9R02-91043,129.0,both
73,City of McFarland,9R02-91110,197.0,both
81,City of Ojai,9R02-91058,231.0,both
99,City of Solvang,9R02-91028,312.0,both


### Filtering out for agencies that appeared in organizations but not vehicles...

In [25]:
df_organizations_only = df_joined1.loc[df_joined1['_merge'] == 'left_only']

In [26]:
df_organizations_only

,agency,ntd_id,itp_id,_merge
0,Access Services,90157,1.0,left_only
1,Alameda-Contra Costa Transit District,90014,4.0,left_only
2,Alpine County,9R02-91116,9.0,left_only
4,Anaheim Transportation Network,90211,14.0,left_only
5,Antelope Valley Transit Authority,90121,16.0,left_only
6,Bishop Paiute Tribe,99268,40.0,left_only
7,Blue Lake Rancheria,99292,42.0,left_only
8,Butte County Association of Governments,90208,47.0,left_only
9,Calaveras County,9R02-91063,50.0,left_only
10,California Department of Transportation,9R02,NaN,left_only


## Joining using vehicles on the left this time
* Still getting a strange even split of 26 agencies are only found in vehicles, 26 are found in both?

In [27]:
df_joined2 = vehicles_crosswalk.merge(organizations_crosswalk.drop_duplicates(), on=['agency','ntd_id'], validate = "m:1", how='left', indicator=True)
df_joined2._merge.value_counts()

left_only     26
both          26
right_only     0
Name: _merge, dtype: int64

In [28]:
df_joined2.sort_values('_merge')

,agency,ntd_id,itp_id,_merge
16,"City of Dixon, dba: Readi-Ride",9R02-91041,NaN,left_only
23,"City of Corcoran, dba: Corcoran Area Transit",9R02-91002,NaN,left_only
27,Calaveras Transit Agency,9R02-99442,NaN,left_only
21,"Mariposa County Transit, dba: Mari-Go",9R02-91082,NaN,left_only
20,"Madera County, dba: Madera County Connection T...",9R02-91005,NaN,left_only
19,"City of Shafter, dba: Shafter Dial-A-Ride",9R02-91120,NaN,left_only
18,"City of Eureka, dba: Eureka Transit Service",9R02-91093,NaN,left_only
17,"County of Siskiyou, dba: Siskiyou County Transit",9R02-91048,NaN,left_only
28,Palo Verde Valley Transit Agency,9R02-99454,NaN,left_only
41,"City of Taft, dba: Taft Area Transit",9R02-91066,NaN,left_only


## Going to try to use fuzzy matcher on this..
* Slightly better results: 38 matches...out of the 52 rural agencies!

* Now I can look at NTD ID to make sure these really are the same agencies.

In [29]:
left_on = ["agency"]
right_on = ["agency"]

In [30]:
vehicles_organization = fuzzymatcher.fuzzy_left_join(vehicles_crosswalk, organizations_crosswalk, left_on, right_on)

In [31]:
vehicles_organization.sort_values('best_match_score')

,best_match_score,__id_left,__id_right,agency_left,ntd_id_left,agency_right,ntd_id_right,itp_id
723,-0.155887,47_left,31_right,"City of Escalon, dba: eTrans",9R02-91078,City of California City,9R02-91111,51.0
528,-0.155887,29_left,31_right,"City of Ridgecrest, dba: Ridgerunner",9R02-91006,City of California City,9R02-91111,51.0
11,-0.142256,7_left,168_right,San Benito County LTA,9R02-91009,San Mateo County Transit District,90009,290.0
104,-0.090304,12_left,10_right,County of Sacramento Municipal Services Agency...,9R02-90216,California Department of Transportation,9R02,NaN
350,-0.087536,21_left,137_right,"Mariposa County Transit, dba: Mari-Go",9R02-91082,Marin County Transit District,90234,194.0
587,-0.049747,37_left,158_right,Modoc Transportation Agency,9R02-91008,Riverside Transit Agency,90031,269.0
779,-0.036144,49_left,31_right,"Wasco, City of",9R02-99426,City of California City,9R02-91111,51.0
480,-0.017965,28_left,158_right,Palo Verde Valley Transit Agency,9R02-99454,Riverside Transit Agency,90031,269.0
586,0.016724,36_left,132_right,Glenn Transit Service,9R02-91088,Lassen Transit Service Agency,9R02-91098,162.0
304,0.036304,19_left,97_right,"City of Shafter, dba: Shafter Dial-A-Ride",9R02-91120,City of Shafter,9R02-91120,303.0


### Filtering out for bad match scores (stuff lower than 0.02) & clearly incorrect agencies with 'high' match scores.

In [32]:
#stuff to filter out before cutting off at scores below 0.02
subset = ['Plumas County Transportation Commission', 'Calaveras Transit Agency', 'Alpine County Local Transportation Commission','Kern Regional Transit','City of Dixon, dba: Readi-Ride','Kern Regional Transit']

In [33]:
#filter out a bunch of stuff all at once
vehicles_organization2 = vehicles_organization[~vehicles_organization.agency_left.isin(subset)]

In [34]:
vehicles_organization2 = vehicles_organization2.loc[vehicles_organization2['best_match_score'] > 0.02]

In [35]:
vehicles_organization2.sort_values('best_match_score')

,best_match_score,__id_left,__id_right,agency_left,ntd_id_left,agency_right,ntd_id_right,itp_id
304,0.036304,19_left,97_right,"City of Shafter, dba: Shafter Dial-A-Ride",9R02-91120,City of Shafter,9R02-91120,303.0
305,0.099742,20_left,136_right,"Madera County, dba: Madera County Connection T...",9R02-91005,Madera County,9R02-91005,188.0
254,0.117530,18_left,51_right,"City of Eureka, dba: Eureka Transit Service",9R02-91093,City of Eureka,9R02-91093,108.0
57,0.144350,9_left,144_right,Nevada County Transit Services,9R02-91095,Nevada County,9R02-91095,221.0
420,0.157668,23_left,39_right,"City of Corcoran, dba: Corcoran Area Transit",9R02-91002,City of Corcoran,9R02-91002,78.0
831,0.157668,50_left,79_right,"City of Needles, dba: Needles Area Transit",9R02-91020,City of Needles,9R02-91020,220.0
613,0.157668,41_left,102_right,"City of Taft, dba: Taft Area Transit",9R02-91066,City of Taft,9R02-91066,330.0
203,0.171887,17_left,174_right,"County of Siskiyou, dba: Siskiyou County Transit",9R02-91048,Siskiyou County,9R02-91048,83.0
667,0.252084,43_left,31_right,City of California City,9R02-91111,City of California City,9R02-91111,51.0
673,0.254328,46_left,35_right,"City of Chowchilla, dba: Chowchilla Area Transit",9R02-91071,City of Chowchilla,9R02-91071,65.0


### Prepping Crosswalk to look at 5311 Black Cat

In [36]:
#cleaning up
vehicles_organization3 = vehicles_organization2.drop(columns=['best_match_score', 'agency_right', 'ntd_id_right','__id_left','__id_right','best_match_score'])

In [37]:
vehicles_organization3 = vehicles_organization3.rename(columns = {'agency_left':'agency'})

In [38]:
len(vehicles_organization3)

38

## 5311 Data Crosswalk Test
* Comparing the agency names in 5311 against Vehicle Data to ensure everything is the same using fuzzy matcher https://pbpython.com/record-linking.html

In [39]:
left_on = ["organization_name"]
right_on = ["agency"]

In [40]:
vehicles_organization3.head(2)

,agency,ntd_id_left,itp_id
0,Fresno County Rural Transit Agency,9R02-91007,117.0
1,Eastern Sierra Transit Authority,9R02-91062,99.0


In [41]:
matched_results = fuzzymatcher.fuzzy_left_join(crosswalk_5311, vehicles_organization3, left_on, right_on)

### Looking at best match score...
* Only 35 matches....
* Either use dictionary or merge black cat back in
* Filter out for left only in organization crosswalk

In [42]:
#stuff to filter out before cutting off at scores...
subset2 = ['Plumas County Transportation Commission', 'Imperial County Transportation Commission', 'Glenn County Transportation Commission','Tehama County Transit Agency','Tuolumne County Transit Agency (TCTA)']

In [43]:
#filter out a bunch of stuff all at once
matched_results2 = matched_results[~matched_results.organization_name.isin(subset2)]

In [44]:
#filter out for anything with a score below 0.208 which cuts off at Eureka Transit Service
matched_results2 = matched_results2.loc[matched_results2['best_match_score'] > 0.06 ]

In [45]:
len(matched_results2)

35

In [46]:
matched_results2.sort_values('organization_name')

,best_match_score,__id_left,__id_right,organization_name,agency,ntd_id_left,itp_id
837,0.087900,77_left,11_right,Amador Transit,Amador Regional Transit System,9R02-91000,11.0
24,0.281294,4_left,29_right,City of Arcata,City of Arcata,9R02-91018,18.0
103,0.281294,12_left,19_right,City of Arvin,City of Arvin,9R02-91027,21.0
104,0.281294,13_left,23_right,City of Auburn,City of Auburn,9R02-91032,23.0
105,0.281294,14_left,33_right,City of California City,City of California City,9R02-91111,51.0
0,0.210125,0_left,35_right,City of Chowchilla,"City of Chowchilla, dba: Chowchilla Area Transit",9R02-91071,65.0
838,0.374528,78_left,18_right,City of Corcoran - Corcoran Area Transit,"City of Corcoran, dba: Corcoran Area Transit",9R02-91002,78.0
25,0.281294,5_left,24_right,City of Dinuba,City of Dinuba,9R02-91040,93.0
163,0.281294,18_left,30_right,City of Guadalupe,City of Guadalupe,9R02-91043,129.0
164,0.281294,19_left,37_right,City of McFarland,City of McFarland,9R02-91110,197.0


### Looking at the agencies that had super low match scores

In [48]:
#filter out for anything with a score below 0.208 which cuts off at Eureka Transit Service
matched_results.loc[matched_results['best_match_score'] <  0.06 ]

47